In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from edbo.plus.optimizer_botorch import EDBOplus
import os
import json
os.chdir('/Users/kate_fieseler/PycharmProjects/edboplus_scope/test/thompson_SNAr/figure_S2_train_test')

expression symbols from pyomo.core.expr  (deprecated in 6.6.2) (called from
<frozen importlib._bootstrap>:219)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/kate_fieseler/PycharmProjects/edboplus_scope/test/thompson_SNAr/figure_S2_train_test'

#### Artificially recreating figure S2. How long will it take to find the maximum peak height? What does the chemical space look like for each round?
I am using data from Thompson + Cooks SNAr HTE experimentation.

## 1. Creating a search scope using EDBO+.

##### To run EDBO+ we need to first create a reaction scope (search space) in a .csv format with all the possible combinations that you want to consider for our optimization. 
##### You can "manually" create a reaction scope using any spreadsheet editor (such as Excel, Libreoffice Calc, ...) but we have also created a tool to help you generating combinatorial search spaces. 
##### For instance, lets say that we want to consider 3 solvents ($\bf{THF}$, $\bf{Toluene}$, $\bf{DMSO}$), 4 temperatures ($\bf{-10}$, $\bf{0}$, $\bf{10}$, $\bf{25}$) and 3 concentration levels ($\bf{0.1}$, $\bf{0.2}$, $\bf{1.0}$). We can introduce these in the EDBO+ scope generator in a dictionary form as follows:

In [3]:

# NOTE: Only using R1-B12 (D) since it means there was a double addition in the final product and had more positive results. There is no difference in the R1_B12 (S) and (D) reactants, just the product. 
reaction_components = {
    'A': ['R1-A1', 'R1-A2', 'R1-A3', 'R1-A4', 'R1-A5', 'R1-A6', 'R1-A7', 'R1-A8'],
    'B': ['R1-B1', 'R1-B2', 'R1-B3', 'R1-B4', 'R1-B5', 'R1-B6', 'R1-B7', 'R1-B8', 'R1-B9', 'R1-B10', 'R1-B11', 'R1-B12 (D)'],
    'solvent': ['NMP', '1,4-dioxane'],
    'base': ['DIPEA', 'TEA', 'NaOtBu', 'None'],
    'T': [20, 150],
}

with open('/Users/kate_fieseler/PycharmProjects/edboplus_scope/test/thompson_SNAr/figure_S2/reaction_components.json', 'w') as f:
    json.dump(reaction_components, f)

##### Now we need to pass the previous dictionary to the $\bf{generate\_reaction\_scope}$ tool in the EDBOplus class.

In [46]:
EDBOplus().generate_reaction_scope(
    components=reaction_components, 
    filename='SNAr_optimization.csv',
    check_overwrite=False
)

Generating reaction scope...


,A,B,solvent,base,T
0,R1-A1,R1-B1,NMP,DIPEA,20
1,R1-A1,R1-B1,NMP,DIPEA,150
2,R1-A1,R1-B1,NMP,TEA,20
3,R1-A1,R1-B1,NMP,TEA,150
4,R1-A1,R1-B1,NMP,NaOtBu,20
...,...,...,...,...,...
1531,R1-A8,R1-B12 (D),"1,4-dioxane",TEA,150
1532,R1-A8,R1-B12 (D),"1,4-dioxane",NaOtBu,20
1533,R1-A8,R1-B12 (D),"1,4-dioxane",NaOtBu,150
1534,R1-A8,R1-B12 (D),"1,4-dioxane",None,20


##### We can always load/read the previously generated reaction scope using any spreadsheet editor but in this case we will use Pandas for that:

In [47]:
import pandas as pd
df_scope = pd.read_csv('SNAr_optimization.csv')  # Load csv file.

##### Now we can check the number of combinations in the reaction scope:

In [48]:
n_combinations = len(df_scope)
print(f"Your reaction scope has {n_combinations} combinations.")

Your reaction scope has 1536 combinations.


##### Adding training data. 

In [73]:
tests = []
for i in range(5):
    exp = pd.read_csv(f'figure_S2_test_{i+1}.csv')
    # fill in all A, A_smiles, B, B_smiles before other values
    df_reactants_filled = df_scope.merge(exp[['A', 'A_smiles', 'B', 'B_smiles']], on=['A', 'B'], how='left')
    df_edbo = df_reactants_filled.merge(exp[['A', 'B', 'solvent', 'base', 'T', 'peak_height']], on=['A', 'B', 'solvent', 'base', 'T',], how='left')
    print(df_edbo)
    df_edbo = df_edbo.fillna('PENDING')
    df_edbo = df_edbo.drop_duplicates()
    df_edbo.to_csv(f'edbo_initial_figure_S2_test_{i+1}.csv', index=False)
    tests.append(df_edbo)

          A           B      solvent   base    T             A_smiles  \
0     R1-A1       R1-B1          NMP  DIPEA   20  N[C@@H]1CCCC[C@H]1N   
1     R1-A1       R1-B1          NMP  DIPEA   20  N[C@@H]1CCCC[C@H]1N   
2     R1-A1       R1-B1          NMP  DIPEA   20  N[C@@H]1CCCC[C@H]1N   
3     R1-A1       R1-B1          NMP  DIPEA  150  N[C@@H]1CCCC[C@H]1N   
4     R1-A1       R1-B1          NMP  DIPEA  150  N[C@@H]1CCCC[C@H]1N   
...     ...         ...          ...    ...  ...                  ...   
4923  R1-A8  R1-B12 (D)  1,4-dioxane   None   20     c1ccc2[nH]cnc2c1   
4924  R1-A8  R1-B12 (D)  1,4-dioxane   None   20     c1ccc2[nH]cnc2c1   
4925  R1-A8  R1-B12 (D)  1,4-dioxane   None  150     c1ccc2[nH]cnc2c1   
4926  R1-A8  R1-B12 (D)  1,4-dioxane   None  150     c1ccc2[nH]cnc2c1   
4927  R1-A8  R1-B12 (D)  1,4-dioxane   None  150     c1ccc2[nH]cnc2c1   

                        B_smiles  peak_height  
0              CC(=O)c1ccc(F)cc1          NaN  
1              CC(=O)c1ccc(

In [71]:
tests

[          A           B      solvent   base    T             A_smiles  \
 0     R1-A1       R1-B1          NMP  DIPEA   20  N[C@@H]1CCCC[C@H]1N   
 1     R1-A1       R1-B1          NMP  DIPEA   20  N[C@@H]1CCCC[C@H]1N   
 2     R1-A1       R1-B1          NMP  DIPEA   20  N[C@@H]1CCCC[C@H]1N   
 3     R1-A1       R1-B1          NMP  DIPEA  150  N[C@@H]1CCCC[C@H]1N   
 4     R1-A1       R1-B1          NMP  DIPEA  150  N[C@@H]1CCCC[C@H]1N   
 ...     ...         ...          ...    ...  ...                  ...   
 4923  R1-A8  R1-B12 (D)  1,4-dioxane   None   20     c1ccc2[nH]cnc2c1   
 4924  R1-A8  R1-B12 (D)  1,4-dioxane   None   20     c1ccc2[nH]cnc2c1   
 4925  R1-A8  R1-B12 (D)  1,4-dioxane   None  150     c1ccc2[nH]cnc2c1   
 4926  R1-A8  R1-B12 (D)  1,4-dioxane   None  150     c1ccc2[nH]cnc2c1   
 4927  R1-A8  R1-B12 (D)  1,4-dioxane   None  150     c1ccc2[nH]cnc2c1   
 
                         B_smiles peak_height  
 0              CC(=O)c1ccc(F)cc1     PENDING  
 1            

## 2a. First steps, initializing EDBO+ with training data. OHE.

In [57]:
EDBOplus().run(
    filename='figure_S2_test_1.csv',  # Previously generated scope.
    objectives=['peak_height'],  # Objectives to be optimized.
    objective_mode=['max'],  # Maximize yield and ee but minimize side_product.
    batch=96,  # Number of experiments in parallel that we want to perform in this round.
    columns_features=['A', 'B', 'solvent', 'base', 'T', 'peak_height'], # features to be included in the model.
    init_sampling_method='cvtsampling'  # initialization method.
)

The following columns are categorical and will be encoded using One-Hot-Encoding: ['A', 'B', 'solvent', 'base']


AttributeError: Can only use .str accessor with string values!

##### EDBO+ has created a column for each objective and added $\bf{PENDING}$ values to all of them so you can track the experiments that you have been collecting during the optimization campaign.
##### We can also see that EDBO+ has created a new $\bf{priority}$ column. This column is used to distinguish between high and low priority samples. The top entries (with $\bf{priority=1}$) highlight the next suggested samples.

##### We can check now the first 96 experiments in the scope by reading the $\bf{my\_optimization.csv}$ file:

In [1]:
df_edbo = pd.read_csv('OHE_SNAr_optimization.csv')
df_edbo

NameError: name 'pd' is not defined

## 3. Adding training data in EDBO+.

##### Note: We will use Python and Pandas to add new training data in this example. But you can always edit and add new data into the '.csv' file using any spreedsheet editor (such as Excel, Libreoffice Calc, ...) if that's more convinient for you.

##### Let's open again the $\bf{my\_optimization.csv}$ file we generated before:

In [55]:
df_edbo = pd.read_csv('OHE_SNAr_optimization.csv')
df_edbo.head(5)

,A,B,solvent,base,T,peak_height,priority
0,R1-A6,R1-B11,"1,4-dioxane",None,150,PENDING,1
1,R1-A5,R1-B10,NMP,None,20,PENDING,1
2,R1-A3,R1-B12 (D),"1,4-dioxane",None,150,PENDING,1
3,R1-A6,R1-B6,"1,4-dioxane",NaOtBu,150,PENDING,1
4,R1-A7,R1-B6,NMP,DIPEA,20,PENDING,1


##### We can fill the entries with the round 1 data from figure_S2 in paper:

In [52]:
exp = pd.read_csv('figure_S2_tidy.csv')
df_edbo = df_edbo.merge(exp[['A', 'B', 'solvent', 'base', 'T', 'peak_height']], on=['A', 'B', 'solvent', 'base', 'T'], how='left', suffixes=['', '_observed'])
df_edbo = df_edbo.drop('peak_height', axis=1).rename(columns={'peak_height_observed': 'peak_height'})
df_edbo = df_edbo.fillna('PENDING')

##### We can check that we have filled out the first entry with our "observed data":

In [53]:
df_edbo

,A,B,solvent,base,T,priority,peak_height
0,R1-A6,R1-B3,NMP,None,150,1,34.8
1,R1-A8,R1-B2,"1,4-dioxane",None,20,1,319.7
2,R1-A7,R1-B8,NMP,NaOtBu,20,1,211.1
3,R1-A1,R1-B1,NMP,DIPEA,20,0,31.7
4,R1-A6,R1-B3,"1,4-dioxane",NaOtBu,20,0,70.1
...,...,...,...,...,...,...,...
1019,R1-A3,R1-B6,"1,4-dioxane",TEA,20,0,107.6
1020,R1-A3,R1-B6,"1,4-dioxane",TEA,150,0,571.2
1021,R1-A3,R1-B6,"1,4-dioxane",NaOtBu,20,0,29.0
1022,R1-A3,R1-B6,"1,4-dioxane",NaOtBu,150,0,21.4


In [54]:
df_edbo.to_csv('SNAr_optimization_round0.csv', index=False)

## 4. Running EDBO+ with training data.

##### First let's check our previous data (which include some $\bf{yield}$, $\bf{ee}$ and $\bf{side\_product}$ observations, which will be used to train the model):

In [55]:
df_edbo_round0 = pd.read_csv('SNAr_optimization_round0.csv')
df_edbo_round0.head(5)

,A,B,solvent,base,T,priority,peak_height
0,R1-A6,R1-B3,NMP,None,150,1,34.8
1,R1-A8,R1-B2,"1,4-dioxane",None,20,1,319.7
2,R1-A7,R1-B8,NMP,NaOtBu,20,1,211.1
3,R1-A1,R1-B1,NMP,DIPEA,20,0,31.7
4,R1-A6,R1-B3,"1,4-dioxane",NaOtBu,20,0,70.1


##### Now that we have introduced some "observations" in our $\bf{my\_optimization\_round0.csv}$ file, we can execute EDBO+ to suggest samples using these "observations" as training data.

In [34]:
EDBOplus().run(
    filename='SNAr_optimization_round0.csv',  # Previously generated scope (including observations).
    objectives=['peak_height'],  # Objectives to be optimized.
    objective_mode=['max'],  # Maximize yield and ee but minimize side_product.
    batch=3,  # Number of experiments in parallel that we want to perform in this round.
    columns_features='all', # features to be included in the model.
    init_sampling_method='cvtsampling'  # initialization method.
)

The following columns are categorical and will be encoded using One-Hot-Encoding: ['A', 'B', 'solvent', 'base']
Using EHVI acquisition function.
Using hyperparameters optimized for continuous variables.
Number of QMC samples using SobolQMCNormalSampler sampler: 128
Acquisition function optimized.
Predictions obtained and expected improvement obtained.


,A,B,solvent,base,T,time,priority,peak_height
2714,R1-A7,R1-B7,NMP,None,200,15,1.0,PENDING
2739,R1-A7,R1-B7,NMP,DIPEA,200,15,1.0,PENDING
2749,R1-A7,R1-B7,NMP,DIPEA,200,4,1.0,PENDING
2088,R1-A8,R1-B8,NMP,TEA,200,15,0.0,PENDING
2068,R1-A8,R1-B8,NMP,TEA,200,4,0.0,PENDING
...,...,...,...,...,...,...,...,...
4058,R1-A1,R1-B1,"1,4-dioxane",None,20,0,-1.0,30.0
4073,R1-A1,R1-B1,"1,4-dioxane",NaOtBu,150,15,-1.0,34.2
4056,R1-A1,R1-B1,"1,4-dioxane",NaOtBu,20,0,-1.0,86.5
4049,R1-A1,R1-B1,"1,4-dioxane",DIPEA,150,15,-1.0,192.9


##### Again the samples suggested by EDBO+ have $\bf{priority = +1}$. In addition, we asign $\bf{priority = -1}$ to the experiments that we have already run (these are at the bottom of the dataset).

## Extra: Accessing the model predictions.

##### Each time that EDBO+ is executed with training data it will generate a .csv file with the predictions for the entire scope (including the 'untested' samples).
##### In the previous example, by running EDBO+ with training data, we generated two files: $\bf{my\_optimization\_round0.csv}$ and a second file with the predictions $\bf{pred\_my\_optimization\_round0.csv}$.
##### Let's have a look to the predictions file using Pandas:

In [45]:
df_predictions_round0 = pd.read_csv('pred_my_optimization_round0.csv')
df_predictions_round0.style.background_gradient(subset=['priority'], cmap='plasma')
df_predictions_round0

,solvent,T,concentration,yield,ee,side_product,priority,yield_predicted_mean,yield_predicted_variance,yield_expected_improvement,ee_predicted_mean,ee_predicted_variance,ee_expected_improvement,side_product_predicted_mean,side_product_predicted_variance,side_product_expected_improvement
0,Toluene,25,0.1,PENDING,PENDING,PENDING,1.0,35.389211,106.283951,77.555477,25.010862,106.997267,78.086238,0.149964,0.356658,0.426849
1,Toluene,0,0.1,PENDING,PENDING,PENDING,1.0,35.329193,104.724311,76.285865,25.071282,105.427159,76.865090,0.149762,0.351424,0.422797
2,DMSO,-10,1.0,PENDING,PENDING,PENDING,1.0,35.398475,106.347990,77.610820,25.001535,107.061735,78.133148,0.149995,0.356872,0.427030
3,Toluene,25,1.0,PENDING,PENDING,PENDING,0.0,35.389097,106.282733,77.554453,25.010976,106.996040,78.085316,0.149963,0.356653,0.426846
4,Toluene,25,0.2,PENDING,PENDING,PENDING,0.0,35.389187,106.283698,77.555264,25.010886,106.997011,78.086046,0.149964,0.356657,0.426849
5,Toluene,10,1.0,PENDING,PENDING,PENDING,0.0,34.341670,23.938856,12.172579,26.065433,24.099519,13.059380,0.146449,0.080332,0.250805
6,Toluene,10,0.2,PENDING,PENDING,PENDING,0.0,34.383269,27.152537,14.630186,26.023555,27.334769,15.530646,0.146588,0.091116,0.253990
7,Toluene,10,0.1,PENDING,PENDING,PENDING,0.0,34.393614,27.974466,15.263226,26.013140,28.162214,16.166053,0.146623,0.093874,0.254927
8,Toluene,0,1.0,PENDING,PENDING,PENDING,0.0,35.328145,104.684494,76.253682,25.072337,105.387074,76.833685,0.149759,0.351290,0.422695
9,Toluene,0,0.2,PENDING,PENDING,PENDING,0.0,35.328975,104.716042,76.279182,25.071502,105.418835,76.858569,0.149762,0.351396,0.422776


##### In the previous dataset we can access the model predictions ($\bf{predicted\_mean}$ and $\bf{predicted\_variance}$ columns) but also to the expected improvement values (in the $\bf{expected\_improvement}$ columns) for each objective ($\bf{yield}$, $\bf{ee}$ and $\bf{side\_product}$.).